In [6]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
#import inflect #numeric/cardinal

In [7]:
#!pip install TextBlob

In [8]:
dirPath = r"/data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"

path = dirPath + "/" + reviewPath

In [9]:
import os
current = os.getcwd()
current
path = current + path
path

'/home/tone/Study/NLP/data/reviews_Tools_and_Home_Improvement_5.json.gz'

In [10]:
dataFile = gzip.open(path, 'r')

In [11]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [12]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [13]:
len(productReviews)

134476

In [14]:
#productReviews[101718]

In [15]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [16]:
productsReviewData.shape
productsReviewData.columns
productsReviewData.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A4IL0CLL27Q33,104800001X,D. Brennan,"[0, 1]","I hate it when my shirt collars, not otherwise...",5.0,Perfect for collar stay management,1390953600,"01 29, 2014"
1,A3Q5W5E7TDVLJF,104800001X,funnyc130,"[0, 0]",These little magnets are really powerful for t...,5.0,Neat,1369958400,"05 31, 2013"


In [17]:
reduceData = productsReviewData.loc[:, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]

In [18]:
reduceData = reduceData.loc[reduceData['overall']!=3.0, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]

In [19]:
rating = lambda x: 1 if x > 2.0 else 0

In [20]:
reduceData['rating'] = reduceData['overall'].apply(rating)
unfavorableReviews = list(reduceData.loc[ reduceData['rating'] == 0, 
                                         ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').index)
#unfavorableReviews
reduceData.loc[ reduceData['rating'] == 0, ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').head(5)


,reviewText,helpful,overall,rating
134322,Amazing that they designed this thing without ...,"[2, 2]",1.0,0
55236,I was ready to pick up one of these at a woodw...,"[30, 38]",1.0,0
55234,I bought this set from amazon but is not like ...,"[8, 15]",1.0,0
105852,Horrible.Not one blade runs true. Some are as...,"[0, 0]",1.0,0
55216,This product is almost a joke. The case is to...,"[4, 9]",1.0,0


In [21]:
reviewTextProcessing = pd.DataFrame(reduceData.reviewText.copy())

In [22]:
reviewTextProcessing['rating'] = reduceData['rating']
reviewTextProcessing['overall'] = reduceData['overall']

In [23]:
productsReviewData.dtypes
len(productsReviewData)
# reviewText, rating, overall
reviewTextProcessing.shape

(123707, 3)

In [24]:
#reviewTextProcessing.loc[reviewTextProcessing['rating']==0, ['rating', 'overall', 'Processed', 'SentimentModel']]

In [25]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

#print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)


In [26]:
sample1 = "s***"
print(re.sub("\*", "#", sample1))

sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)
expand_contractions(sample1)

s###


'shit'

In [27]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        review[count] = " ".join(eachSent)
    wholeText = [" ".join(review) ]
    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

#paragraphText, wholeText = singleReviewCleanUp(sample)
#wholeText, paragraphText
#paragraphText

In [28]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [29]:
%%time
reviewTextProcessing['Processed'] = reviewTextProcessing.reviewText.apply(cleanUpTextParagraph)


CPU times: user 1min 46s, sys: 326 ms, total: 1min 46s
Wall time: 1min 46s


In [30]:
#sample = " I don't know what to do. I need to test through tough. Do you have any suggestions"
#result = cleanUpTextParagraph(sample)
#result

In [31]:
from textblob import TextBlob

# need to do a data check first
def getPolaritySubjectivity(data):
    #print(data)
    paragraphText = data
    sumPolarity = []
    sumSubjectivity = []
    for each in paragraphText:
        sumPolarity.append(TextBlob(each).sentiment.polarity)
        sumSubjectivity.append(TextBlob(each).sentiment.subjectivity)
    #print("sumPol", sumPolarity)
    return sumPolarity, sumSubjectivity
    #return sum(sumPolarity), sum(sumSubjectivity)

    #print(each, TextBlob(each).sentiment.polarity, TextBlob(each).sentiment.subjectivity)
#(sample2, TextBlob(sample2).sentiment.polarity, TextBlob(sample2).sentiment.subjectivity)


In [32]:
#Return the sume of polarity in a paragraph sentence when give a text in a list form of sentence

getPolarityFromText = lambda text: getPolaritySubjectivity(text)[0]

In [33]:
#for each in sampleText:
#    sample = cleanUpTextParagraph(each)
#    print(sample, getPolarityFromText(sample))

In [34]:
#print(stopWords)

In [35]:
#sentiment = lambda text: TextBlob(text).sentiment.polarity

In [36]:
adjustSentimentModel = lambda x: [sum(x), 0.0] if sum(x) < 0 else [sum(x), 1.0]

In [37]:
%%time
reviewTextProcessing['Sentiment']= reviewTextProcessing.Processed.apply(getPolarityFromText)

CPU times: user 3min 35s, sys: 424 ms, total: 3min 36s
Wall time: 3min 36s


In [38]:
%%time
reviewTextProcessing['SentimentModel']= reviewTextProcessing.Sentiment.apply(adjustSentimentModel)

CPU times: user 60 ms, sys: 14.3 ms, total: 74.4 ms
Wall time: 73.7 ms


In [39]:
#reviewTextProcessing.loc[unfavorableReviews, ['Processed', 'Sentiment', 'SentimentModel']]
#reviewTextProcessing.dtypes

In [40]:
reviewTextProcessing['CleanedJoin'] = reviewTextProcessing['Processed'].apply(', '.join)


In [41]:
reviewTextProcessing.shape
reviewTextProcessing.columns

Index(['reviewText', 'rating', 'overall', 'Processed', 'Sentiment',
       'SentimentModel', 'CleanedJoin'],
      dtype='object')

In [42]:
#from gensim import corpora
#dictionary = corpora.Dictionary(T)
#print(dictionary.token2id)

In [43]:
reviewTextProcessing['CleanedJoin'].head(5)

0    i hate it when my shirt collars not otherwise ...
1    these little magnets are really powerful for t...
2    i wanted something this small to mount on the ...
3    i use these to magnetize my warhammer k miniat...
4    they are soo freaking annoying, why, you spend...
Name: CleanedJoin, dtype: object

In [44]:
reviewTextProcessing.sort_values(by='rating', inplace=True)

In [389]:
reviewTextProcessing.head(2)

,reviewText,rating,overall,Processed,Sentiment,SentimentModel,CleanedJoin
99410,Bought one and loved it! Ordered another two.O...,0,1.0,"[bought one and loved it, ordered another two ...","[0.7, 0.2857142857142857, 3.700743415417188e-1...","[0.9857142857142857, 1.0]","bought one and loved it, ordered another two o..."
79663,What a waste of money. Brand new and worse th...,0,1.0,"[what a waste of money, brand new and worse th...","[-0.2, 0.03712121212121211, 0.0, 0.31785714285...","[0.36709956709956704, 1.0]","what a waste of money, brand new and worse tha..."


In [46]:
size = reviewTextProcessing.loc[reviewTextProcessing['rating'] == 0, :].shape[0]
size

10105

In [47]:
reviewTextProcessing = reviewTextProcessing.iloc[0:size*5, :]


In [70]:
additionalNegatives = reviewTextProcessing.iloc[0:size, :].copy()
additionalNegatives.head(2)

,reviewText,rating,overall,Processed,Sentiment,SentimentModel,CleanedJoin
99410,Bought one and loved it! Ordered another two.O...,0,1.0,"[bought one and loved it, ordered another two ...","[0.7, 0.2857142857142857, 3.700743415417188e-1...","[0.9857142857142857, 1.0]","bought one and loved it, ordered another two o..."
79663,What a waste of money. Brand new and worse th...,0,1.0,"[what a waste of money, brand new and worse th...","[-0.2, 0.03712121212121211, 0.0, 0.31785714285...","[0.36709956709956704, 1.0]","what a waste of money, brand new and worse tha..."


In [388]:
#Pad more negative reviews for better training purposes
testSample = pd.concat([reviewTextProcessing, additionalNegatives])
#testSample.reset_index(drop=True, inplace=True)
testSample.head(1)

,reviewText,rating,overall,Processed,Sentiment,SentimentModel,CleanedJoin
99410,Bought one and loved it! Ordered another two.O...,0,1.0,"[bought one and loved it, ordered another two ...","[0.7, 0.2857142857142857, 3.700743415417188e-1...","[0.9857142857142857, 1.0]","bought one and loved it, ordered another two o..."


In [50]:

import itertools

largeText = testSample.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

60630

In [51]:
#sentiment = reviewTextProcessing['SentimentModel'].to_list()

In [52]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

num_words = 15000


In [53]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
X = largeText
y = testSample['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=310)

x_train_text = X_train
x_test_text = X_test

x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [56]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(45472, 15158, 45472, 15158)

In [57]:
#x_train_tokens[1], x_train_text[1]

In [58]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [59]:
np.mean(num_tokens), np.max(num_tokens)


(122.61286491835725, 4925)

In [60]:
np.std(num_tokens)

143.06143605914914

In [61]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


408

In [62]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.9587168068612898

In [63]:
pad = 'pre'
#pad = 'post'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape, x_test_pad.shape


((45472, 408), (15158, 408))

In [64]:
for i in range(0, 1):
    print(x_train_pad[i])


[ 3065    15     1   139    26     4   260  4527    19     4    25     4
     7     3  3209  4644    45   321   482  2861     1  8293   482  4727
    46     1   575    49   542  3065    12   399     1  5710   353    31
    10  9121   289   282     3  2687    85     8    13  6322   482     4
    25     6   139    38    81   358   793   259    64   100    26    28
    37    10   210    43    48    54     3   473    49   282     8   563
     6    13    25    13     7    85    28  9001  7978    40    38    81
   358     1   358    20    10    50    10   669   100    19    28    20
    10    50    10  1044    45     1   769   358     8     1   575  7020
    14    31  1347    38   358   139   913     1  5710   528     8     1
  5910  7020    66    16   105   139   913   113     1   575  5910  1404
    49 11914    46  1443     5    52    39     4   271     5   139   913
     9   247    24  8522     5   455    19    32    14    74    10    45
  9209    58  1681   461    28    20   100  1219   

In [65]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [66]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [67]:
x_train_text[1]


'i bought the toto toilet toto seat along with this seal, the seal did not fit my plumbing toilet connection, i did not use this seal, it is in the garage in the box'

In [68]:
tokens_to_string(x_train_tokens[1])


'i bought the toto toilet toto seat along with this seal the seal did not fit my plumbing toilet connection i did not use this seal it is in the garage in the box'

In [71]:
#!pip install keras

     |████████████████████████████████| 337kB 4.7MB/s eta 0:00:01


In [72]:
#import keras

Using TensorFlow backend.


In [73]:
from keras import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import GlobalMaxPooling1D
from keras.optimizers import Adam

In [ ]:
import tensorflow as tf
tf.__version__


In [74]:
optimizer = Adam(lr=1e-3)


In [75]:
model_cnn = Sequential()
parameters = Embedding(num_words, 200, input_length=max_tokens)
model_cnn.add(parameters)
model_cnn.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model_cnn.summary()


W0823 21:41:26.307747 139999309981504 deprecation_wrapper.py:119] From /home/tone/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0823 21:41:26.313521 139999309981504 deprecation_wrapper.py:119] From /home/tone/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0823 21:41:26.318373 139999309981504 deprecation_wrapper.py:119] From /home/tone/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0823 21:41:26.381014 139999309981504 deprecation_wrapper.py:119] From /home/tone/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0823 21:41:26.3

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 408, 200)          3000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 407, 100)          40100     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 3,066,213
Trainable params: 3,066,213
Non-trainable params: 0
_________________________________________________________________


In [76]:
index = len(x_train_pad)
index
len(y_train)

45472

In [183]:
%%time
model_cnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=36)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 43198 samples, validate on 2274 samples
Epoch 1/3
43198/43198 [==============================] - 49s 1ms/step - loss: 0.0570 - acc: 0.9784 - val_loss: 0.2200 - val_acc: 0.9384
Epoch 2/3
43198/43198 [==============================] - 49s 1ms/step - loss: 0.0221 - acc: 0.9925 - val_loss: 0.2816 - val_acc: 0.9305
Epoch 3/3
43198/43198 [==============================] - 50s 1ms/step - loss: 0.0156 - acc: 0.9943 - val_loss: 0.3298 - val_acc: 0.9296
CPU times: user 15min 19s, sys: 26.1 s, total: 15min 45s
Wall time: 2min 28s


In [78]:
#more negative training
#model_cnn.fit(x_negative_pad, y_negative, validation_split=0.05, epochs=3, batch_size=36)

In [184]:
#%%time
result = model_cnn.evaluate(x_test_pad, y_test)

15158/15158 [==============================] - 3s 175us/step


In [185]:
print("Accuracy: {0:.2%}".format(result[1]))


Accuracy: 92.96%


In [186]:
#%%time
#y_pred = model_cnn_02.predict(x=x_test_pad[0:1000])
y_pred = model_cnn.predict(x=x_test_pad)
y_pred = y_pred.T[0]



In [187]:
pd.DataFrame(y_pred).describe()

,0
count,15158.000000
mean,0.632692
std,0.468342
min,0.000000
25%,0.000425
50%,0.999588
75%,1.000000
max,1.000000


In [ ]:
#y_pred = y_pred.T[0]
#list(np.where(y_pred.T[0]))

In [188]:
y_pred.shape

(15158,)

In [189]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])


In [190]:
len(cls_pred),

(15158,)

In [191]:
accuracy = (cls_pred==y_test).mean()
accuracy

0.9296081277213353

In [192]:
sample = len(cls_pred)
sample

15158

In [193]:
from sklearn.metrics import confusion_matrix

In [194]:
#Duplicate has been 
tn, fp, fn, tp = confusion_matrix(y_test, cls_pred).ravel()
tn, fp, fn, tp

(4772, 286, 781, 9319)

In [95]:
len(np.where(y_test==0)[0]),len(np.where(y_test==1)[0])

(5058, 10100)

In [96]:
len(np.where(y_train==0)[0]),len(np.where(y_train==1)[0])

(15152, 30320)

In [107]:
cls_true = np.array(y_test)

In [359]:
y_test.index
y_test.values
indexTable = pd.DataFrame()
indexTable['Index'] = y_test.index
indexTable['TrueRating'] = y_test.values
indexTable['CNNRating'] = cls_pred

In [360]:
indexTable.shape

(15158, 3)

In [361]:
compare = indexTable.drop_duplicates(subset="Index")
compare.shape

(14531, 3)

In [375]:
df = testSample.loc[compare.Index.to_list(), :]
df = df.reset_index()
#A.drop_duplicates()
df= df.drop_duplicates(subset="index")
df.rename(columns={"index": "Index"}, inplace=True)
df = df.loc[:, ['Index', 'reviewText', 'overall', 'SentimentModel']]

textBlobRating = lambda result: result[1]
df['textBlobRating'] = df.SentimentModel.apply(textBlobRating)
df.shape

(14531, 5)

In [376]:
result = indexTable.drop_duplicates(subset="Index")
#result.sort_values(by="Index")
result.shape, result.columns

((14531, 3), Index(['Index', 'TrueRating', 'CNNRating'], dtype='object'))

In [377]:
# Here is the result of comparison between TextBlob and CNN
# Merged based On Index
CNNResult = pd.merge(df, result, on="Index", how="inner")
#CNNresult['CNNRating'].astype(int)
CNNResult.columns

Index(['Index', 'reviewText', 'overall', 'SentimentModel', 'textBlobRating',
       'TrueRating', 'CNNRating'],
      dtype='object')

In [378]:
CNNResult.CNNRating = CNNResult.CNNRating.astype(int)
CNNResult.textBlobRating = CNNResult.textBlobRating.astype(int)

In [381]:
CNNResult.columns

Index(['Index', 'reviewText', 'overall', 'SentimentModel', 'textBlobRating',
       'TrueRating', 'CNNRating'],
      dtype='object')

In [383]:
from sklearn.metrics import confusion_matrix

y_true = CNNResult.TrueRating.to_list()
cnn_pred = CNNResult.CNNRating.to_list()
textBlob_pred = CNNResult.textBlobRating.to_list()

In [385]:
tn, fp, fn, tp = confusion_matrix(y_true, cnn_pred).ravel()
tn, fp, fn, tp

(4269, 162, 781, 9319)

In [386]:
tn, fp, fn, tp = confusion_matrix(y_true, textBlob_pred).ravel()
tn, fp, fn, tp

(1303, 3128, 622, 9478)

In [390]:
len(y_true)

14531

In [ ]:
pd.options.display.max_colwidth = 500

#productsReviewData.cleanedText.head(2)